In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

# --- CONFIGURATION ---
# 🚨 IMPORTANT: Replace this with the ACTUAL PATH to your dataset folder.
DATA_DIR = 'train_segmented/train_segmented' 
MODEL_FILE_NAME = 'weed_vs_crop_classifier.h5'

IMAGE_SIZE = (128, 128)  # Must match the input size for prediction
BATCH_SIZE = 32
EPOCHS = 15 # Start with 10-15 epochs; increase if accuracy is still improving.

# --- STEP 1: DATA PREPARATION (Keras Generators) ---

print("1. Setting up Data Generators...")

# Initialize the generator with normalization and data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values (0 to 1)
    validation_split=0.2,       # Use 20% of data for validation
    
    # Data Augmentation (Crucial for a robust model)
    rotation_range=25,          
    width_shift_range=0.1,      
    height_shift_range=0.1,     
    zoom_range=0.15,
    horizontal_flip=True,       
    fill_mode='nearest'         
)

# Training Data Generator
try:
    train_generator = datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    # Validation Data Generator
    validation_generator = datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    class_names = list(train_generator.class_indices.keys())
    NUM_CLASSES = len(class_names)

    if train_generator.n == 0:
         raise ValueError(f"Found 0 images in the training subset. Check your DATA_DIR path: {DATA_DIR}")
    
    print(f"\nDetected {NUM_CLASSES} Classes: {class_names}")
    print(f"Total Training Images: {train_generator.n}")
    print(f"Total Validation Images: {validation_generator.n}")

except Exception as e:
    print(f"\n--- FATAL ERROR IN DATA LOADING ---")
    print("Ensure the path is correct and the folders contain images. Details:")
    print(e)
    exit() 

# --- STEP 2: BUILD THE CNN MODEL ---

def create_cnn_model(input_shape, num_classes):
    """Creates a moderately sized Convolutional Neural Network (CNN)."""
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        
        # Block 2
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        # Classification Head
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5), # Regularization to prevent overfitting
        Dense(num_classes, activation='softmax') # Output layer
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_cnn_model(IMAGE_SIZE + (3,), NUM_CLASSES)
# model.summary() # Uncomment to see the model architecture

# --- STEP 3: TRAIN AND SAVE THE MODEL ---

print(f"\n2. Starting Model Training for {EPOCHS} Epochs...")

history = model.fit(
    train_generator,
    epochs=EPOCHS, 
    validation_data=validation_generator
)

print(f"\nTraining Complete. Model saved as '{MODEL_FILE_NAME}'")
model.save(MODEL_FILE_NAME)

1. Setting up Data Generators...
Found 3803 images belonging to 12 classes.
Found 947 images belonging to 12 classes.

Detected 12 Classes: ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
Total Training Images: 3803
Total Validation Images: 947


C:\Users\91965\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



2. Starting Model Training for 15 Epochs...


C:\Users\91965\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 0.2445 - loss: 2.1946 - val_accuracy: 0.4182 - val_loss: 1.6124
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 61s 511ms/step - accuracy: 0.4289 - loss: 1.6167 - val_accuracy: 0.5734 - val_loss: 1.1932
Epoch 3/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.5744 - loss: 1.1939 - val_accuracy: 0.6864 - val_loss: 0.9864
Epoch 4/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.6317 - loss: 1.0719 - val_accuracy: 0.6727 - val_loss: 0.9751
Epoch 5/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.6429 - loss: 1.0484 - val_accuracy: 0.7392 - val_loss: 0.8062
Epoch 6/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.6784 - loss: 0.9404 - val_accuracy: 0.7212 - val_loss: 0.8210
Epoch 7/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.6807 - loss: 0.9179 - val_accuracy: 0.7497 - val_loss: 0.7670
Epoch 8/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 62s 523ms/step - accuracy: 0.7232 - loss: 0.8214 - val_


Training Complete. Model saved as 'weed_vs_crop_classifier.h5'


In [13]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

# --- CONFIGURATION ---
# IMPORTANT: This must match the name used when you saved the model after training.
MODEL_FILE_NAME = 'weed_vs_crop_classifier.h5' 
IMAGE_SIZE = (128, 128) # Must match the size used during training
CAMERA_INDEX = 0        # 0 is usually the built-in webcam

# --- STEP 1: DEFINE CLASS NAMES and WEED/CROP MAPPING ---

# The full list of 12 classes from the Plant Seedlings dataset.
# The order MUST match the internal index order used by your trained model.
# NOTE: The index order is typically determined alphabetically by Keras.
# It is strongly recommended to check the exact order from your training script's output!
CLASS_NAMES = [
    'Black-grass',
    'Charlock',
    'Cleavers',
    'Common Chickweed',
    'Common wheat',      # <-- CROP
    'Fat Hen',
    'Loose Silky-bent',
    'Maize',             # <-- CROP
    'Scentless Mayweed',
    'Shepherds Purse',
    'Small-flowered Cranesbill',
    'Sugar beet'         # <-- CROP
]

# Define which specific classes are the main CROPS (the plants you want to keep)
CROP_SPECIES = ['Common wheat', 'Maize', 'Sugar beet']


# --- STEP 2: LIVE PREDICTION FUNCTION ---

def classify_frame_weed_or_crop(frame, model, class_names, target_size):
    """
    Classifies an image frame into one of the 12 detailed classes, 
    then maps the result to the binary CROP or WEED action.
    """
    
    # 1. Preprocessing (Must match training steps)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    img_resized = cv2.resize(img_rgb, target_size)
    img_array = np.array(img_resized) / 255.0  
    img_array = np.expand_dims(img_array, axis=0) 

    # 2. Make Prediction (on 12 classes)
    predictions = model.predict(img_array, verbose=0)
    probabilities = tf.nn.softmax(predictions[0]) 
    
    predicted_index = np.argmax(probabilities)
    confidence = np.max(probabilities) * 100
    predicted_class = class_names[predicted_index]
    
    # 3. Binary Mapping and Action
    is_crop = predicted_class in CROP_SPECIES
    
    if is_crop:
        action_class = "CROP (Main Plant) 🌱"
        action_guidance = "ACTION: KEEP. This is a valuable crop seedling."
        color = (0, 255, 0) # Green
    else:
        action_class = "WEED (Unwanted Plant) ⚠️"
        action_guidance = "ACTION: REMOVE. This is a competing weed species."
        color = (0, 0, 255) # Red

    # 4. Compile Result
    result = (
        f"Detailed ID: {predicted_class}\n"
        f"Final Classification: {action_class}\n"
        f"Confidence: {confidence:.2f}%\n"
        f"Guidance: {action_guidance}"
    )
    
    return predicted_class, confidence, action_class, color, result

# --- STEP 3: CAMERA CAPTURE LOOP ---

def run_weed_detection_system():
    # Load the trained model
    try:
        model = load_model(MODEL_FILE_NAME)
    except FileNotFoundError:
        print(f"\n--- ERROR: Model Not Found ---")
        print(f"Please ensure '{MODEL_FILE_NAME}' exists and was trained successfully.")
        return

    cap = cv2.VideoCapture(CAMERA_INDEX) 
    if not cap.isOpened():
        print("\n--- ERROR ---: Cannot open camera. Check hardware.")
        return

    print("\n--- WEED/CROP CLASSIFIER READY ---")
    print("Press 'c' to Capture and Classify the current frame.")
    print("Press 'q' to Quit.")

    # Variables for displaying the last result
    last_action = "Awaiting Capture..."
    last_confidence = 0.0
    text_color = (255, 255, 255) # White default

    while True:
        ret, frame = cap.read()
        if not ret: break

        display_frame = frame.copy()
        
        # Display current status/last prediction
        text = f"LAST: {last_action} | Conf: {last_confidence:.2f}%"
        cv2.putText(display_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
        cv2.putText(display_frame, "Press 'c' to CAPTURE", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        cv2.imshow('Live Weed/Crop Analysis', display_frame)

        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        
        elif key == ord('c'):
            print("\n[INFO] Capturing and classifying...")
            
            # Run the prediction
            _, confidence, action_class, color, result_text = classify_frame_weed_or_crop(frame, model, CLASS_NAMES, IMAGE_SIZE)
            
            # Update the displayed text variables
            last_action = action_class
            last_confidence = confidence
            text_color = color

            # Log the result
            print(result_text)
            
            # Brief pause
            cv2.waitKey(200) 

    cap.release()
    cv2.destroyAllWindows()

# Execute the live classifier
run_weed_detection_system()


--- WEED/CROP CLASSIFIER READY ---
Press 'c' to Capture and Classify the current frame.
Press 'q' to Quit.

[INFO] Capturing and classifying...


Detailed ID: Charlock
Final Classification: WEED (Unwanted Plant) ⚠️
Confidence: 12.91%
Guidance: ACTION: REMOVE. This is a competing weed species.
